In [50]:
import pandas as pd
import numpy as np
import re
from pyvi import ViTokenizer
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, SimpleRNN
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

## Load data

In [51]:
train = pd.read_excel(r'Data\data_aivivn.xlsx')

In [52]:
count_sentiment = pd.value_counts(train['Label'])
count_sentiment

0    8882
1    7205
Name: Label, dtype: int64

## Pre-processing review

In [53]:
def pre_processing(text):
    
    # chuyển về chữ thường
    text = text.lower()
    
    # Xóa icon
    emoji_pattern = re.compile("["
                   u"\U0001F600-\U0001F64F"  # emoticons
                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                   u"\U00002702-\U000027B0"
                   u"\U000024C2-\U0001F251"
                   u"\U0001f926-\U0001f937"
                   u"\u200d"
                   u"\u2640-\u2642" 
                   "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Chuan hoa tieng viet, tieng anh, thuat ngu
    replace_list={
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
    'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
    'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
    'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
    'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
    #Chuẩn hóa 1 số sentiment words/English words
    ':))': '  positive ', ':)': ' positive ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
    'okey': ' ok ', 'ôkê': ' ok ', ' oki ': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
    ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ', 'cam on':u'cám ơn',u'cảm ơn':'cám ơn',
    ' not ': u' không ',' khoong ': ' không ', u' kg ': u' không ','ko ': 'không', ' k ': u' không ',' kh ':u' không ',' kô ':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', ' ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',' k ':u' không ',u'chẳng':u'không',u'đéo':u'không',
    'he he': ' positive ','hehe': ' positive ','hihi': ' positive ', 'haha': ' positive ', 'hjhj': ' positive ',
    ' lol ': ' negative ',' cc ': ' negative ','cute': u' dễ thương ','huhu': ' negative ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', ' j ': u' gì ', '“': ' ',
    ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
    'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' positive ', 'store': u' cửa hàng ',
    'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
    'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
    'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
    'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
    'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
    'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
    'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
    'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ', ' quả ng ': ' quảng ',
    'dep': u' đẹp ',u'xầu':u'xấu',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u' qủa': u' quả', 
    ' iu ': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' positive ', 
    ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
    'qc':u'quảng cáo',u' éo ':' negative ',' bik ':' biết ',
    #dưới 3* quy về 1*, trên 3* quy về 5*
    '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ', '5 sao': '5star',
    'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ', '3 sao':'5star','4 sao':'5star','1 sao':'1star',
    '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ','1*':' 1star ', '2*':' 1star ','3*':' 5star ','4*':' 5star ','5*':' 5star '
    }
    for k, v in replace_list.items():
        text = text.replace(k, v)

    #Xóa dấu câu
    text = re.sub('['+string.punctuation+']', ' ', text)

    # Tokenizer
    text = ViTokenizer.tokenize(text)
    return text

In [54]:
New_Sentence = []
for i in range(train.shape[0]):
    New_Sentence.append(pre_processing(train['Sentence'][i]))
train['New_Sentence'] = New_Sentence

## Split data

In [55]:
from sklearn.model_selection import train_test_split

In [56]:
X_train, X_test, y_train, y_test = train_test_split(train.loc[:,'New_Sentence'].values, train.loc[:,'Label'].values, test_size=0.2, random_state=0)

In [57]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(12869,)
(12869,)
(3218,)
(3218,)


## Embedding

In [64]:
vocab_size = 10000
embedding_dim = 64
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [59]:
# Tạo Tokenizer
tokenizer = Tokenizer(num_words= vocab_size, oov_token= oov_tok)

# Đưa từ vào tokenizer để tạo từ điển
tokenizer.fit_on_texts(X_train)

# Biến các câu train thành sequences
X_train = tokenizer.texts_to_sequences(X_train)

# Padding cho các train sequences
X_train = pad_sequences(X_train, maxlen= max_length, padding=padding_type, truncating= trunc_type)

# Biến các câu test thành sequences tương ứng
X_test = tokenizer.texts_to_sequences(X_test)

X_test = pad_sequences(X_test, maxlen= max_length, padding=padding_type, truncating= trunc_type)

In [60]:
print(X_train.shape)
print(X_test.shape)

(12869, 100)
(3218, 100)


## Build Model

### Simple RNN

In [71]:
model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_dim))
model1.add(SimpleRNN(32))
model1.add(Dense(16, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))

In [72]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          640000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                3104      
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 643,649
Trainable params: 643,649
Non-trainable params: 0
_________________________________________________________________


In [73]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [74]:
model1.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
403/403 [==============================] - 11s 27ms/step - loss: 0.6341 - accuracy: 0.6261 - val_loss: 0.5783 - val_accuracy: 0.6917
Epoch 2/10
403/403 [==============================] - 9s 23ms/step - loss: 0.5157 - accuracy: 0.7233 - val_loss: 0.6117 - val_accuracy: 0.6336
Epoch 3/10
403/403 [==============================] - 10s 24ms/step - loss: 0.4574 - accuracy: 0.7835 - val_loss: 0.5563 - val_accuracy: 0.7045
Epoch 4/10
403/403 [==============================] - 12s 30ms/step - loss: 0.4252 - accuracy: 0.8110 - val_loss: 0.5563 - val_accuracy: 0.7237
Epoch 5/10
403/403 [==============================] - 13s 32ms/step - loss: 0.4131 - accuracy: 0.8214 - val_loss: 0.5263 - val_accuracy: 0.7505
Epoch 6/10
403/403 [==============================] - 11s 28ms/step - loss: 0.4368 - accuracy: 0.8113 - val_loss: 0.6828 - val_accuracy: 0.5864
Epoch 7/10
403/403 [==============================] - 12s 29ms/step - loss: 0.5947 - accuracy: 0.6857 - val_loss: 0.5800 - val_accuracy: 

### LSTM

In [82]:
model2 = Sequential()
model2.add(Embedding(vocab_size, embedding_dim, input_length=X_train.shape[1]))
model2.add(LSTM(32))
model2.add(Dense(16, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          640000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                3104      
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 643,649
Trainable params: 643,649
Non-trainable params: 0
_________________________________________________________________


In [83]:
model1.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
403/403 [==============================] - 9s 24ms/step - loss: 0.1846 - accuracy: 0.9420 - val_loss: 0.4416 - val_accuracy: 0.8428
Epoch 2/5
403/403 [==============================] - 9s 22ms/step - loss: 0.1721 - accuracy: 0.9444 - val_loss: 0.4197 - val_accuracy: 0.8505
Epoch 3/5
403/403 [==============================] - 10s 24ms/step - loss: 0.1279 - accuracy: 0.9608 - val_loss: 0.4350 - val_accuracy: 0.8564
Epoch 4/5
403/403 [==============================] - 9s 23ms/step - loss: 0.1300 - accuracy: 0.9601 - val_loss: 0.4109 - val_accuracy: 0.8599
Epoch 5/5
403/403 [==============================] - 9s 23ms/step - loss: 0.1139 - accuracy: 0.9639 - val_loss: 0.4952 - val_accuracy: 0.8415
